In [1]:
addprocs(CPU_CORES)

@everywhere qs = 1.0 + 2.0.^(-5:2)
times = Array{Float64}(length(qs),4)
means = Array{Float64}(length(qs),4)

@everywhere begin
  using StochasticDiffEq, DiffEqBase, DiffEqProblemLibrary,
        Plots, LaTeXStrings, ParallelDataTransfer, DiffEqMonteCarlo
  srand(99 + myid())
  full_prob = oval2ModelExample(largeFluctuations=true,useBigs=false)
  prob = deepcopy(full_prob)
  prob.tspan=(0.0,1.0)
  println("Solve once to compile.")
  sol = solve(prob,EM(),dt=1/2^(18))
  Int(sol.u[end][1]!=NaN)
  println("Compilation complete.")
  num_runs = 10000

  probs = Vector{SDETestProblem}(3)
  p1 = Vector{Any}(3)
  p2 = Vector{Any}(3)
  p3 = Vector{Any}(3)
  ## Problem 1
  probs[1] = prob_sde_linear
  ## Problem 2
  probs[2] = prob_sde_wave
  ## Problem 3
  probs[3] = prob_sde_additive
end
println("Setup Complete")

## Timing Runs

@everywhere function runAdaptive(i,k)
  sol = solve(prob,SRIW1(),dt=1/2^(8),abstol=2.0^(-14),reltol=2.0^(-9),maxIters=Int(1e12),qmax=qs[k])
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end

#Compile
test_mc = monte_carlo_simulation(probs[1],SRIW1(),dt=1/2^(4),adaptive=true,numMonte=1000,abstol=2.0^(-1),reltol=0,adaptiveAlg=:RSwM3)
calculate_sim_errors(test_mc)

  likely near In[1]:1
)WARNING: Method definition solve(DiffEqBase.AbstractSDEProblem{#uType<:Any, #tType<:Any, #isinplace<:Any, #NoiseClass<:Any, #F<:Any, #F2<:Any, #F3<:Any}, #algType<:DiffEqBase.AbstractSDEAlgorithm, Any, Any) in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10 overwritten in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10.
 in module StochasticDiffEqWARNING: Method definition solve(DiffEqBase.AbstractSDEProblem{#uType<:Any, #tType<:Any, #isinplace<:Any, #NoiseClass<:Any, #F<:Any, #F2<:Any, #F3<:Any}, #algType<:DiffEqBase.AbstractSDEAlgorithm, Any, Any, Any) in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10 overwritten in module StochasticDiffEq at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:10.
 at /home/crackauc/.julia/v0.5/StochasticDiffEq/src/solve.jl:47WARNING: Method definition solve(DiffEqBase.AbstractSDEProblem{#uType<:Any, #tType

DiffEqMonteCarlo.MonteCarloTestSimulation{Array{DiffEqBase.SDETestSolution{Array{Float64,1},Array{Float64,1},Float64,Array{Float64,1},StochasticDiffEq.LinearInterpolationData{Array{Float64,1},Array{Float64,1}},Array{Float64,1},DiffEqBase.SDETestProblem{Float64,Float64,false,false,:Diagonal,DiffEqProblemLibrary.##23#24,DiffEqProblemLibrary.##25#26,Base.Random.#randn},StochasticDiffEq.SRIW1{StochasticDiffEq.RSWM{:RSwM3,Float64}}},1},Float64}(DiffEqBase.SDETestSolution{Array{Float64,1},Array{Float64,1},Float64,Array{Float64,1},StochasticDiffEq.LinearInterpolationData{Array{Float64,1},Array{Float64,1}},Array{Float64,1},DiffEqBase.SDETestProblem{Float64,Float64,false,false,:Diagonal,DiffEqProblemLibrary.##23#24,DiffEqProblemLibrary.##25#26,Base.Random.#randn},StochasticDiffEq.SRIW1{StochasticDiffEq.RSWM{:RSwM3,Float64}}}[DiffEqBase.SDETestSolution{Array{Float64,1},Array{Float64,1},Float64,Array{Float64,1},StochasticDiffEq.LinearInterpolationData{Array{Float64,1},Array{Float64,1}},Array{Floa

## qmax test on Oval2 Model

In [2]:
for k in eachindex(qs)
  ParallelDataTransfer.sendto(workers(), k=k)
  @everywhere srand(99 + myid())
  adaptiveTime = @elapsed numFails = sum(pmap((i)->runAdaptive(i,k),1:num_runs))
  println("k was $k. The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")
  times[k,4] = adaptiveTime
end

k was 1. The number of Adaptive Fails is 0. Elapsed time was 55.851457279
k was 2. The number of Adaptive Fails is 0. Elapsed time was 48.069866667
k was 3. The number of Adaptive Fails is 0. Elapsed time was 44.240459253
k was 4. The number of Adaptive Fails is 0. Elapsed time was 47.46605247
k was 5. The number of Adaptive Fails is 0. Elapsed time was 54.595354196
k was 6. The number of Adaptive Fails is 0. Elapsed time was 57.797899981
k was 7. The number of Adaptive Fails is 0. Elapsed time was 57.335670469
k was 8. The number of Adaptive Fails is 0. Elapsed time was 61.550315689


## qmax test on other problems

In [3]:
@everywhere num_runs*=10
for k in eachindex(probs)
  println("Problem $k")
  ## Setup
  prob = probs[k]
  ParallelDataTransfer.sendto(workers(), prob=prob)

  for i in eachindex(qs)
    ParallelDataTransfer.sendto(workers(), i=i)
    msim = monte_carlo_simulation(prob,dt=1/2^(4),SRIW1(),adaptive=true,numMonte=num_runs,abstol=2.0^(-13),reltol=0,adaptiveAlg=:RSwM3,qmax=qs[i])
    test_msim = calculate_sim_errors(msim)
    times[i,k] = test_msim.elapsedTime
    means[i,k] = test_msim.error_means[:final]
    println("for k=$k and i=$i, we get that the error was $(means[i,k]) and it took $(times[i,k]) seconds")
  end
end

Problem 1
for k=1 and i=1, we get that the error was 0.00041341121640084725 and it took 4.205403582 seconds
for k=1 and i=2, we get that the error was 0.0004273420002984963 and it took 2.952503536 seconds
for k=1 and i=3, we get that the error was 0.0004372564369346694 and it took 2.640535315 seconds
for k=1 and i=4, we get that the error was 0.00043847050483064625 and it took 4.054253131 seconds
for k=1 and i=5, we get that the error was 0.00042354141632578785 and it took 2.489509917 seconds
for k=1 and i=6, we get that the error was 0.00043401533351849846 and it took 2.860503635 seconds
for k=1 and i=7, we get that the error was 0.0004352515473584442 and it took 7.440884024 seconds
for k=1 and i=8, we get that the error was 0.000425675142663137 and it took 8.707556047 seconds
Problem 2
for k=2 and i=1, we get that the error was 6.061217819853804e-8 and it took 2.816913742 seconds
for k=2 and i=2, we get that the error was 1.0018152569563065e-7 and it took 2.051193846 seconds
for k=2 